<h1>Windows Virtual Desktop with Azure AD Domain Services</h1>

<h2>This document was created to guide someone to creating a Windows Virtual Desktop with an instance of Azure AD Domain Services.</h2>

The Windows Virtual Desktop will be created with FSLogix for the user profiles.  Those profiles will be pointed to an Azure File Share on premium storage.


<h3>Connect to Azure</h3>



In [1]:
# Connect to Azure
if (get-module -list | where name -match "az"){
    Write-Host "The Az module is installed"
    Write-Host "Connecting to Azure with the Az module"
    connect-azaccount
} else {
    Write-Host "The Az module is NOT installed"
    Write-Host "Installing Az module"
    Install-Module Az
    Write-Host "Importing AzureAD module"
    Import-Module Az
    Write-Host "Connecting to Azure with the Az module"
    connect-azaccount
}


The below script needs to be able to find the current output cell; this is an easy method to get it.

The Az module is installed
Connecting to Azure with the Az module

Account                SubscriptionName TenantId                             Environment
-------                ---------------- --------                             -----------
carlos.mccray@live.com Pay-As-You-Go    33477442-d92d-4d17-aef5-57aa410384c1 AzureCloud



<h3>Create a Resource Group</h3>

In [2]:
# Create a resource group
$global:rgName = 'rg-coreinfrastructure'
$global:region = 'eastus'
try {
    $rgExists = Get-AzResourceGroup -Name $rgName
} finally {}

if ($rgExists -ne $null){
    Write-Host "The resource group has already been created"
} else {
    New-AzResourceGroup -Name $rgName -Location $region
}


Get-AzResourceGroup: 
Line |
   5 |      $rgExists = Get-AzResourceGroup -Name $rgName
     |                  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | 8:50:37 AM - Provided resource group does not exist.

ResourceGroupName : rg-coreinfrastructure
Location          : eastus
ProvisioningState : Succeeded
Tags              : 
ResourceId        : /subscriptions/8348c69f-7f45-46be-8513-7cb4762c69c2/resourceGroups/rg-coreinfra
                    structure




This infrastructure within this resource group will include...

    - An Azure Active Directory Domain Services domain
    - A virtual network
    - A subnet for the AADDS and one for WVD
    - A management VM for the WVD

<h3>Create the virtual network and subnets</h3>

In [3]:
$global:vnetName = "core-vnet"

# Create the dedicated subnet for Azure AD Domain Services.
$global:subnetName = "aadds-subnet"
$global:aaddsSubnet = New-AzVirtualNetworkSubnetConfig `
  -Name $subnetName `
  -AddressPrefix 10.0.100.0/24

# Create an additional subnet for your own VM workloads
$global:workloadSubnet = New-AzVirtualNetworkSubnetConfig `
  -Name workloads-subnet `
  -AddressPrefix 10.0.110.0/24
  
# Create an additional subnet for your own VM workloads
$global:WVDSubnet = New-AzVirtualNetworkSubnetConfig `
  -Name wvd-subnet `
  -AddressPrefix 10.0.120.0/24

# Create the virtual network in which you will enable Azure AD Domain Services.
$vnetExists = Get-AzVirtualNetwork -Name $vnetName
if ($vnetExists -ne $null){
    Write-Host "The virtual network already exists."
} else {
    $vnet = New-AzVirtualNetwork -ResourceGroupName $rgName -Location $region -Name $vnetName -AddressPrefix 10.0.0.0/16 -Subnet $aaddsSubnet,$workloadSubnet,$WVDSubnet
}

Update Property Name
Cmdlet invocation changes :
    Old Way : -ResourceId
    New Way : -NatGatewayId
Update Property Name
Cmdlet invocation changes :
    Old Way : -InputObject
    New Way : -NatGateway
Note : Go to https://aka.ms/azps-changewarnings for steps to suppress this breaking change warning, and other information on breaking changes in Azure PowerShell.
Update Property Name
Cmdlet invocation changes :
    Old Way : -ResourceId
    New Way : -NatGatewayId
Update Property Name
Cmdlet invocation changes :
    Old Way : -InputObject
    New Way : -NatGateway
Note : Go to https://aka.ms/azps-changewarnings for steps to suppress this breaking change warning, and other information on breaking changes in Azure PowerShell.
Update Property Name
Cmdlet invocation changes :
    Old Way : -ResourceId
    New Way : -NatGatewayId
Update Property Name
Cmdlet invocation changes :
    Old Way : -InputObject
    New Way : -NatGateway
Note : Go to https://aka.ms/azps-changewarnings for steps t

<h3>Create the AADDS into the Resource Group</h3> 

We will create a service principal account and then add it to the AAD DC Administrators group.
We will then register the Azure AD Domain Services resource provider.

<h4>Add the Posh Test Gallery Repository</h4>

This is so that you can use the AzureAD module on Powershell Core.  It is not supported in the current version for Powershell

In [4]:
Write-Host "Adding the Posh Test Gallery for the AzureAD that can be used with Powershell Core"
$packageSource = Get-PackageSource -Name 'Posh Test Gallery'

if (!$packageSource)
{
	$packageSource = Register-PackageSource -Trusted -ProviderName 'PowerShellGet' -Name 'Posh Test Gallery' -Location 'https://www.poshtestgallery.com/api/v2/'
}

Adding the Posh Test Gallery for the AzureAD that can be used with Powershell Core


<h4>Import the AzureAD Module and Connect</h4>

In [5]:
# Check if module is installed
$module = Get-Module 'AzureAD.Standard.Preview' -ListAvailable -ErrorAction SilentlyContinue

if (!$module) 
{
  Write-Host "Installing module AzureAD.Standard.Preview ..."
  $module = Install-Module -Name 'AzureAD.Standard.Preview' -Force -Scope CurrentUser -SkipPublisherCheck -AllowClobber 
  Write-Host "Module installed"
}

# Module doesn't automatically load after install - need to import explictly for Pwsh Core
Import-Module -Name $module.RootModule

$id = (Get-AZTenant).id
Connect-AzureAD -tenantid $id


Account                Environment TenantId                             TenantDomain       AccountT
                                                                                           ype
-------                ----------- --------                             ------------       --------
carlos.mccray@live.com AzureCloud  33477442-d92d-4d17-aef5-57aa410384c1 mccrayprojects.com User



<h3>Create the New Service Principal and AAD DC Administrators Group</h3>

In [6]:
# Create the service principal for Azure AD Domain Services
try {
    Write-Host "Creating service principal for Azure AD DS"
    New-AzureADServicePrincipal -AppId "2565bd9d-da50-47d4-8b85-4c97f669dc36"
} finally {}

# First, retrieve the object ID of the 'AAD DC Administrators' group.
$GroupObjectId = Get-AzureADGroup -Filter "DisplayName eq 'AAD DC Administrators'" | Select-Object ObjectId

# If the group doesn't exist, create it
if (!$GroupObjectId) {
    Write-Host "Creating AAD DC Administrators group"
    $GroupObjectId = New-AzureADGroup -DisplayName "AAD DC Administrators" `
    -Description "Delegated group to administer Azure AD Domain Services" `
    -SecurityEnabled $true `
    -MailEnabled $false `
    -MailNickName "AADDCAdministrators"
  }
else {
  Write-Output "Admin group already exists."
}

# Retrieve the object ID of the user you'd like to add to the group.
$UserObjectId = Get-AzureADUser `
  -Filter "UserPrincipalName eq 'carlos.mccray_live.com#EXT#@carlosmccraylive.onmicrosoft.com'" | Select-Object ObjectId

# Add the user to the 'AAD DC Administrators' group.
$members = Get-AzADGroup -ObjectId $groupobjectid.objectid | get-azadgroupmember

if($members -eq $null){
    Write-Host "Add user to the AAD DC Administrators group"
    Get-AzADGroup -ObjectId $groupobjectid.objectid | Add-AzADGroupMember -memberobjectid $userobjectid.objectid
} else {
    Write-Host "User is in group"
}


# First, register the Azure AD Domain Services resource provider
Write-Host "Register Microsoft.AD provider"
Register-AzResourceProvider -ProviderNamespace Microsoft.AAD



Creating service principal for Azure AD DS

Creating AAD DC Administrators group
Add user to the AAD DC Administrators group
Register Microsoft.AD provider
ObjectId                             AppId                                DisplayName
--------                             -----                                -----------
d0bd0f53-4542-4e32-8561-5b7a70f2b2e6 2565bd9d-da50-47d4-8b85-4c97f669dc36 Domain Controller Servic…

ProviderNamespace : Microsoft.AAD
RegistrationState : Registered
ResourceTypes     : {DomainServices, DomainServices/oucontainer, locations, 
                    locations/operationresults…}
Locations         : {West US, Central US, East US, South Central US…}
ZoneMappings      : 




<h3>Create the NSG for Azure AD Domain Services</h3>

In [7]:
$NSGName = "aaddsNSG"

# Create a rule to allow inbound TCP port 3389 traffic from Microsoft secure access workstations for troubleshooting
$nsg201 = New-AzNetworkSecurityRuleConfig -Name AllowRD `
    -Access Allow `
    -Protocol Tcp `
    -Direction Inbound `
    -Priority 201 `
    -SourceAddressPrefix CorpNetSaw `
    -SourcePortRange * `
    -DestinationAddressPrefix * `
    -DestinationPortRange 3389

# Create a rule to allow TCP port 5986 traffic for PowerShell remote management
$nsg301 = New-AzNetworkSecurityRuleConfig -Name AllowPSRemoting `
    -Access Allow `
    -Protocol Tcp `
    -Direction Inbound `
    -Priority 301 `
    -SourceAddressPrefix AzureActiveDirectoryDomainServices `
    -SourcePortRange * `
    -DestinationAddressPrefix * `
    -DestinationPortRange 5986

# Create the network security group and rules
$nsg = New-AzNetworkSecurityGroup -Name $NSGName `
    -ResourceGroupName $rgName `
    -Location $region `
    -SecurityRules $nsg201,$nsg301

# Get the existing virtual network resource objects and information
Write-Host "Getting virtual network"
$vnet = Get-AzVirtualNetwork -Name $vnetName -ResourceGroupName $rgName
Write-Host "Getting subnet"
$subnet = Get-AzVirtualNetworkSubnetConfig -VirtualNetwork $vnet -Name $subnetName
$addressPrefix = $subnet.AddressPrefix

# Associate the network security group with the virtual network subnet
Write-Host "Associating NSG with subnet"
Set-AzVirtualNetworkSubnetConfig -Name $subnetName -VirtualNetwork $vnet -AddressPrefix $addressPrefix -NetworkSecurityGroup $nsg
$vnet | Set-AzVirtualNetwork

Getting virtual network
Getting subnet
Associating NSG with subnet
Update Property Name
Cmdlet invocation changes :
    Old Way : -ResourceId
    New Way : -NatGatewayId
Update Property Name
Cmdlet invocation changes :
    Old Way : -InputObject
    New Way : -NatGateway
Note : Go to https://aka.ms/azps-changewarnings for steps to suppress this breaking change warning, and other information on breaking changes in Azure PowerShell.

Name                   : core-vnet
ResourceGroupName      : rg-coreinfrastructure
Location               : eastus
Id                     : /subscriptions/8348c69f-7f45-46be-8513-7cb4762c69c2/resourceGroups/rg-core
                         infrastructure/providers/Microsoft.Network/virtualNetworks/core-vnet
Etag                   : W/"d59c0d7f-4d78-41a3-a80f-10b7ce5409c5"
ResourceGuid           : 7142bb51-90e9-414d-9c01-791fe4e7062f
ProvisioningState      : Succeeded
Tags                   : 
AddressSpace           : {
                           "AddressPrefi

<h3>Create the Managed Domain</h3
    

In [ ]:
$dsModule = Get-Module -Name Az.ADDomainServices
if ($dsModule -eq $nnull){
    Write-Host "Installing AD Domain Services module"
    Install-Module -Name Az.ADDomainServices -repository psgallery -force
} else {
    Write-Host "AD Domain Services module is installed"
}

$AzureSubscriptionId = (Get-AzContext).Subscription.id
$ManagedDomainName = "mccrayprojects.com"

# Enable Azure AD Domain Services for the directory.
$replicaSet = New-AzADDomainServiceReplicaSet -Location $region -SubnetId "/subscriptions/$AzureSubscriptionId/resourceGroups/$rgName/providers/Microsoft.Network/virtualNetworks/$vnetName/subnets/aadds-subnet"

Write-Host "Creating new Azure AD Domain Services domain called $ManagedDomainName"
New-AzADDomainService -Name $ManagedDomainName -ResourceGroupName $rgName -DomainName $ManagedDomainName -ReplicaSet $replicaSet
Write-Host "Your AD Domain Services Domain should be complete."

Installing AD Domain Services module


Install-Package: C:\Users\me\.dotnet\tools\.store\microsoft.dotnet-interactive\1.0.218102\microsoft.dotnet-interactive\1.0.218102\tools\net5.0\any\Modules\PowerShellGet\PSModule.psm1:9685
Line |
9685 |  … talledPackages = PackageManagement\Install-Package @PSBoundParameters
     |                     ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Administrator rights are required to install or update. Log on to the computer
     | with an account that has Administrator rights, and then try again, or install by
     | adding "-Scope CurrentUser" to your command. You can also try running the Windows
     | PowerShell session with elevated rights (Run as Administrator).

Creating new Azure AD Domain Services domain called mccrayprojects.com





<h2>Now your Domain is setup</h2>

